In [24]:
import os
import pandas as pd
import numpy as np
import datetime
import urllib

# Display Data about All Wildfire Incidents in Indonesia
Gather information for all wildfire events in Sumatra

In [25]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [26]:
wf_data  = pd.read_csv("../data-total.csv")
wf_data.columns
# rename columns
wf_data.rename(columns={'Lat': 'lat', 
                        'Long': 'long', 
                        'Satellite': 'satellite', 
                        'Time_UTC': 'time_utc', 
                        'Date': 'date', 
                        'Source': 'source', 
                        'PROVINSI': 'provinsi',
                        'KAB_KOTA': 'kab_kota'}, inplace=True)
# adding '-' to date column values
wf_data.date = pd.to_datetime(wf_data.date, format="%Y%m%d")
wf_data.date = wf_data.date.astype('string')

FileNotFoundError: [Errno 2] No such file or directory: '../data-total.csv'

In [ ]:
# All provinces in sumatera
sumatera_provinces = [
    'SUMATERA UTARA',
    'JAMBI',
    'RIAU',
    'SUMATERA BARAT',
    'SUMATERA SELATAN',
    'BENGKULU',
    'LAMPUNG',
    'ACEH'
]
# create filter so that only events in sumatera will be returned.
sumatera_filter = wf_data['provinsi'].apply(lambda x: x in sumatera_provinces) 
sumatera_only = wf_data.loc[sumatera_filter]

In [ ]:
display(sumatera_only.info())
display(sumatera_only.provinsi.unique())
display(sumatera_only.head())

In [27]:
# changing the date data type to datetime
# Adding date day, month, and year to the data frame


if 'year' not in sumatera_only.columns.values:
    date_formatted = sumatera_only['date'].astype('datetime64[ns]')
    date = pd.DatetimeIndex(sumatera_only['date'])
    sumatera_only.insert(len(sumatera_only.columns), "year", date.year, True) # adding column
    sumatera_only.insert(len(sumatera_only.columns), "month", date.month, True) # adding column
    sumatera_only.insert(len(sumatera_only.columns), "day", date.day, True) # adding column


In [28]:
sumatera_only = sumatera_only.sort_values(by=['date'], 
                                          ascending=True,
                                          ignore_index=True)
sumatera_only


,lat,long,satellite,time_utc,date,source,provinsi,kab_kota,kec2006,desaa2006,nama_kaw,year,month,day
0,1.649680,100.892000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN BANGKO PUSAKO,SUNGAI MENASIB,NaN,2016,4,12
1,1.648530,100.884000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN BANGKO PUSAKO,SUNGAI MENASIB,NaN,2016,4,12
2,1.580030,100.855000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN TANAH PUTIH,BALAM SEMPURNA,NaN,2016,4,12
3,4.272360,97.747800,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,ACEH,ACEH TAMIANG,TAMIANG HULU,BATU BEDULANG,NaN,2016,4,12
4,1.656640,100.891000,LPN-NPP,06:24:35 AM,2016-04-12,LAPAN,RIAU,ROKAN HILIR,KECAMATAN BANGKO PUSAKO,SUNGAI MENASIB,NaN,2016,4,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7834,-4.437215,104.505066,LPN-NPP,12:44:37 PM,2020-09-16,LAPAN,LAMPUNG,WAYKANAN,BLAMBANGAN UMPU,TANJUNG RAYA GIHAM,NaN,2020,9,16
7835,-0.470491,100.017670,LPN-NPP,13:04:01 PM,2020-10-17,LAPAN,SUMATERA BARAT,PADANGPARIAMAN,BATANG GASAN,MALAI BAWAH,NaN,2020,10,17
7836,1.306781,99.927521,LPN-NPP,13:04:01 PM,2020-10-17,LAPAN,SUMATERA UTARA,PADANG LAWAS,BARUMUN TENGAH,SIPAGABU,NaN,2020,10,17
7837,0.785336,99.191719,LPN-NPP,13:04:01 PM,2020-10-17,LAPAN,SUMATERA UTARA,MANDAILINGNATAL,NATAL,PARDAMEAN BARU,NaN,2020,10,17


# Import earth engine API which is called ee


In [29]:
import ee
import folium
from folium import plugins

In [11]:
# Authenticate earth engine servers
ee.Authenticate()

#initialize API
ee.Initialize()

Enter verification code: 4/1AX4XfWjDo1Gpvi1HTnxJkQJaBO0FE8-tKCOn9yiOXu1eFbErYXAbzUIlxsI

Successfully saved authorization token.


In [30]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.ImageCollection(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# cloud masking
def cloudMask(collection_name):
    remove_cloud = collection_name.mask(collection_name.select('QA_PIXEL').eq(2720))
    return remove_cloud

def cloudMask2(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20))\
                .addBands(image.metadata('system:time_start'))



# Satellite Imagery Discovery
Date must be long enough for the satellite to capture the image, e.g. 16 days, 24 days, or evenr 365.

# Landsat "LANDSAT/LC08/C02/T1_TOA"
USGS Landsat 8 Collection 2 Tier 1 TOA Reflectance


In [31]:
####################################################### lANDSAT #################################################################
# Select row column
row = 9

# Set coordinates
coordinates = [float(sumatera_only.lat.values[row]), float(sumatera_only.long.values[row])]
# coordinates = [37.3861, -122.0839]

# filter image collection
# set base date
base_date = ee.Date(sumatera_only.date.values[row])
region = ee.Geometry.Rectangle([coordinates[1]-0.2, coordinates[0]-0.2, 
                                coordinates[1]+0.2, coordinates[0]+0.2]);

# set geometry point
# point = ee.Geometry.Point(float(sumatera_only.long.values[row]), float(sumatera_only.lat.values[row]))
#################################################################################################################################


####################################################### PREFIRE #################################################################
# pre fire
prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

# landsat image pre wildfire event
pre_landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')\
                       .filterDate(prefire_date_start, prefire_date_end)\
                       .filterBounds(region)\
                       .map(cloudMask)\
                       .median()



# Calculate NBR for pre fire
pre_ndvi = pre_landsat_collection.normalizedDifference(['B5', 'B4']).gt(ee.Number(0.1)).rename('NDVI')
pre_landsat_collection = pre_landsat_collection.updateMask(pre_ndvi)
pre_nbr = pre_landsat_collection.normalizedDifference(['B5', 'B7'])
# pre_ndwi = pre_landsat_collection.normalizedDifference(['B3', 'B5']).rename('NDWI')
# pre_mndwi = pre_landsat_collection.normalizedDifference(['B2', 'B5']).rename('MNDWI')
# check_pre = pre_landsat_collection.Not()


#################################################################################################################################


####################################################### POSTFIRE ################################################################
# post fire
postfire_date_start = ee.Date(base_date.advance(0, 'day'))
postfire_date_end = ee.Date(base_date.advance(365, 'day'))

# landsat image post wildfire event
post_landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')\
                       .filterDate(postfire_date_start, postfire_date_end)\
                       .filterBounds(region)\
                       .map(cloudMask)\
                       .median()
    
# post_landsat_collection = post_landsat_collection.qualityMosaic('system:time_start')

# Calculate NBR for post fire
post_ndvi = post_landsat_collection.normalizedDifference(['B5', 'B4']).gt(ee.Number(0.1)).rename('NDVI')
post_landsat_collection = post_landsat_collection.updateMask(post_ndvi)
post_nbr = post_landsat_collection.normalizedDifference(['B5', 'B7'])
# post_ndwi = post_landsat_collection.normalizedDifference(['B2', 'B5']).rename('NDWI')
# post_mndwi = post_landsat_collection.normalizedDifference(['B3', 'B5']).rename('MNDWI')
# post_ndwi_mndwi = post_ndwi.add(post_mndwi).rename('I')


# calculate delta NBR
delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
post_landsat_collection = post_landsat_collection\
                          .addBands(delta_nbr)\
                          .addBands(post_ndvi)
#                           .addBands(post_ndwi)\
#                           .addBands(post_mndwi)\
#                           .addBands(post_ndwi_mndwi)\
#################################################################################################################################


############################################## DISPLAY SATELLITE IMAGE ##########################################################
map_landsat = folium.Map(location=coordinates, zoom_start=11)
folium.Marker(coordinates, popup=sumatera_only.desaa2006).add_to(map_landsat)

# Delta NBR visualization
image_viz_params = {
    'bands': ['DELTA_NBR', 'B5', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(check_pre, image_viz_params, None)
display(map_landsat)

image_viz_params = {
    'bands': ['DELTA_NBR', 'B4', 'B3'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

# image_viz_params = {
#     'bands': ['NDVI'],
#     'min': 0,
#     'max': 0.4,
# }
# map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
# display(map_landsat)

# image_viz_params = {
#     'bands': ['B4'],
#     'min': 0,
#     'max': 0.4,
# }
# map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
# display(map_landsat)

# image_viz_params = {
#     'bands': ['B3'],
#     'min': 0,
#     'max': 0.4,
# }
# map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
# display(map_landsat)

# image_viz_params = {
#     'bands': ['B2'],
#     'min': 0,
#     'max': 0.4,
# }
# map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
# display(map_landsat)

# # Regular Image visualization 
# image_viz_params = {
#     'bands': ['B7', 'B5', 'B3'],
#     'min': 0,
#     'max': 0.4,
# }
# map_landsat.add_ee_layer(pre_landsat_collection, image_viz_params, None)
# display(map_landsat)

# # Regular Image visualization 
# image_viz_params = {
#     'bands': ['B7', 'B5', 'B3'],
#     'min': 0,
#     'max': 0.4,
# }
# map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
# display(map_landsat)
#################################################################################################################################

NameError: name 'check_pre' is not defined

In [39]:
####################################################### lANDSAT #################################################################
# Select row column
row = 4999

# Set coordinates
coordinates = [float(sumatera_only.lat.values[row]), float(sumatera_only.long.values[row])]
# coordinates = [37.3861, -122.0839]

# filter image collection
# set base date
base_date = ee.Date(sumatera_only.date.values[row])
region = ee.Geometry.Rectangle([coordinates[1]-0.15, coordinates[0]-0.15, 
                                coordinates[1]+0.15, coordinates[0]+0.15]);
#################################################################################################################################


####################################################### PREFIRE #################################################################
# pre fire
prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

# landsat image pre wildfire event
pre_landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')\
                       .filterDate(prefire_date_start, prefire_date_end)\
                       .filterBounds(region)\
                       .map(cloudMask2)\
                       .median()

# Calculate NBR for pre fire
pre_nbr = pre_landsat_collection.normalizedDifference(['B5', 'B7'])
#################################################################################################################################


####################################################### POSTFIRE ################################################################
# post fire
postfire_date_start = ee.Date(base_date.advance(1, 'day'))
postfire_date_end = ee.Date(base_date.advance(365, 'day'))

# landsat image post wildfire event
post_landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')\
                       .filterDate(postfire_date_start, postfire_date_end)\
                       .filterBounds(region)\
                       .map(cloudMask2)\
                       .median()


post_nbr = post_landsat_collection.normalizedDifference(['B5', 'B7'])


# calculate delta NBR
delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
post_landsat_collection = post_landsat_collection\
                          .addBands(delta_nbr)
#################################################################################################################################


############################################## DISPLAY SATELLITE IMAGE ##########################################################
map_landsat = folium.Map(location=coordinates, zoom_start=12)
folium.Marker(coordinates, popup=sumatera_only.desaa2006).add_to(map_landsat)

# Delta NBR visualization
image_viz_params = {
    'bands': ['DELTA_NBR', 'B4', 'B3'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

image_viz_params = {
    'bands': ['B7', 'B5', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

image_viz_params = {
    'bands': ['B4'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

image_viz_params = {
    'bands': ['B3'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

image_viz_params = {
    'bands': ['B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)

# Regular Image visualization 
image_viz_params = {
    'bands': ['B7', 'B5', 'B3'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(pre_landsat_collection, image_viz_params, None)
display(map_landsat)

# Regular Image visualization 
image_viz_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)
display(map_landsat)
#################################################################################################################################

# SENTINEL "COPERNICUS/S2"
Sentinel-2 MSI: MultiSpectral Instrument, Level-1C

In [40]:
sumatera_only.loc[2001]


lat                  1.85796
long                 101.696
satellite            LPN-NPP
time_utc         06:24:46 AM
date              2019-02-11
source                 LAPAN
provinsi                RIAU
kab_kota           BENGKALIS
kec2006      KECAMATAN RUPAT
desaa2006        TELUK LECAH
nama_kaw                 NaN
year                    2019
month                      2
day                       11
Name: 2001, dtype: object

# Collect all images and find its median composite image

In [41]:
#################################################################################################################################
# sentinel 2 cloud masking 
def s2CloudMasking(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)\
             .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask)\
                .divide(10000)\
#################################################################################################################################


####################################################### SENTINEL ################################################################
# Select row column
row = 5003


# Set coordinates
coordinates = [float(sumatera_only.lat.values[row]), float(sumatera_only.long.values[row])]
print(coordinates)
region = ee.Geometry.Rectangle([coordinates[1]-0.1, coordinates[0]-0.1, 
                                coordinates[1]+0.1, coordinates[0]+0.1]);


# filter image collection
# set base date
base_date = ee.Date(sumatera_only.date.values[row])

#################################################################################################################################


####################################################### PREFIRE #################################################################
# pre fire
prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

# sentinel image pre wildfire event
pre_sentinel_collection = ee.ImageCollection('COPERNICUS/S2')\
                           .filterDate(prefire_date_start, prefire_date_end)\
                           .filterBounds(region)\
                           .map(s2CloudMasking)\
                           .median()


# Calculate NBR for pre fire
pre_nbr = pre_sentinel_collection.normalizedDifference(['B8', 'B12'])
#################################################################################################################################


####################################################### POSTFIRE ################################################################
# post fire
postfire_date_start = ee.Date(base_date.advance(1, 'day'))
postfire_date_end = ee.Date(base_date.advance(365, 'day'))

# sentinel image post wildfire event
post_sentinel_collection = ee.ImageCollection('COPERNICUS/S2')\
                           .filterDate(postfire_date_start, postfire_date_end)\
                           .filterBounds(region)\
                           .map(s2CloudMasking)\
                           .median()
            

# Calculate NBR for post fire
post_nbr = post_sentinel_collection.normalizedDifference(['B8', 'B12'])
# post_ndwi = post_sentinel_collection.normalizedDifference(['B3', 'B8']).rename('NDWI')

# calculate delta NBR
delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
post_sentinel_collection = post_sentinel_collection.addBands(delta_nbr)
#################################################################################################################################


############################################## DISPLAY SATELLITE IMAGE ##########################################################
map_landsat = folium.Map(location=coordinates, zoom_start=12)
folium.Marker(coordinates, popup=sumatera_only.desaa2006).add_to(map_landsat)

# Delta NBR visualization
image_viz_params = {
    'bands': ['DELTA_NBR', 'B3', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_sentinel_collection, image_viz_params, None)
display(map_landsat)

# # Regular Image visualization 
# image_viz_params = {
#     'bands': ['B12', 'B8', 'B4'],
#     'min': 0,
#     'max': 0.4,
# }
# map_landsat.add_ee_layer(pre_sentinel_collection, image_viz_params, None)
# display(map_landsat)

# # Regular Image visualization 
# image_viz_params = {
#     'bands': ['NDWI', 'B8', 'B4'],
#     'min': 0,
#     'max': 0.4,
# }
map_landsat.add_ee_layer(post_sentinel_collection, image_viz_params, None)
display(map_landsat)
#################################################################################################################################
region = ee.Geometry.Rectangle([coordinates[1]-0.1, coordinates[0]-0.1, 
                                coordinates[1]+0.1, coordinates[0]+0.1]);

# file_path = 'D:\\wildfire-sumatera-dataset\\wildfire-sumatera-geotiff\\sentinel-2\\postfire\\2283'

# os.chdir(file_path)

# download_url = post_sentinel_collection.getDownloadURL({
#     'region': region,
#     'bands' : ['DELTA_NBR'],
#     'scale' : 50,
#     'format': 'GeoTIFF'
# })

# download_path = os.path.join(file_path)
# print(f"download data from {download_url}")
# urllib.request.urlretrieve(download_url,'b8.tif')
# print(f"{download_url} downloaded")
# # import urllib
# file_path = "C:\\Users\\USER\\Downloads\\aa"
# urllib.request.urlretrieve(download_url, file_path)

[-1.70435, 104.007]


In [43]:
#################################################################################################################################
# sentinel 2 cloud masking 
def s2CloudMasking(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)\
             .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask)\
                .divide(10000)\
#################################################################################################################################


####################################################### SENTINEL ################################################################
# Select row column
row = 9


# Set coordinates
coordinates = [float(sumatera_only.lat.values[row]), float(sumatera_only.long.values[row])]
region = ee.Geometry.Rectangle([coordinates[1]-0.1, coordinates[0]-0.1, 
                                coordinates[1]+0.1, coordinates[0]+0.1]);


# filter image collection
# set base date
base_date = ee.Date(sumatera_only.date.values[row])

#################################################################################################################################


####################################################### PREFIRE #################################################################
# pre fire
prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

# sentinel image pre wildfire event
pre_sentinel_collection = ee.ImageCollection('COPERNICUS/S2')\
                           .filterDate(prefire_date_start, prefire_date_end)\
                           .filterBounds(region)\
                           .map(s2CloudMasking)\
                           .median()


# Calculate NBR for pre fire
pre_nbr = pre_sentinel_collection.normalizedDifference(['B8', 'B12'])
#################################################################################################################################


####################################################### POSTFIRE ################################################################
# post fire
postfire_date_start = ee.Date(base_date.advance(1, 'day'))
postfire_date_end = ee.Date(base_date.advance(365, 'day'))

# sentinel image post wildfire event
post_sentinel_collection = ee.ImageCollection('COPERNICUS/S2')\
                           .filterDate(postfire_date_start, postfire_date_end)\
                           .filterBounds(region)\
                           .map(s2CloudMasking)\
                           .median()
            

# Calculate NBR for post fire
post_nbr = post_sentinel_collection.normalizedDifference(['B8', 'B12'])
post_ndwi = post_sentinel_collection.normalizedDifference(['B3', 'B8']).rename('NDWI')

# calculate delta NBR
delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
post_sentinel_collection = post_sentinel_collection.addBands(delta_nbr).addBands(post_ndwi)
#################################################################################################################################


############################################## DISPLAY SATELLITE IMAGE ##########################################################
map_landsat = folium.Map(location=coordinates, zoom_start=12)
folium.Marker(coordinates, popup=sumatera_only.desaa2006).add_to(map_landsat)

# Delta NBR visualization
image_viz_params = {
    'bands': ['DELTA_NBR', 'B3', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_sentinel_collection, image_viz_params, None)
display(map_landsat)

# Regular Image visualization 
image_viz_params = {
    'bands': ['B12', 'B8', 'B4'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(pre_sentinel_collection, image_viz_params, None)
display(map_landsat)

# Regular Image visualization 
image_viz_params = {
    'bands': ['NDWI', 'B8', 'B4'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_sentinel_collection, image_viz_params, None)
display(map_landsat)

# Regular Image visualization 
image_viz_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.4,
}
map_landsat.add_ee_layer(post_sentinel_collection, image_viz_params, None)
display(map_landsat)
#################################################################################################################################